# Работа с геоданными

In [2]:
!pip install folium

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [3]:
import folium
import pandas as pd
import branca.colormap as cm
import json

C:\Users\hp\Anaconda2\lib\site-packages\folium\__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


## Реализация вспомогательных методов

Для отрисовки сетки на карте потребуется описание сетки в формате json 

In [4]:
# Загрузка данных по регионам
region=pd.read_csv('regions.csv',sep=';')

# Фуникция возвращает форматированную пару координат:
# [40.748817, -73.985428]
def coord_to_str(longitude,latitude):
    return '['+str(longitude)+","+str(latitude)+']'

# Функция принимает ячейку region и возвращает 4 пары координат ячейки
def region_to_str(region):
    res=''
    delim=''
    for longitude,latitude in zip ([region.west,region.west,region.east,region.east],[region.south,region.north,region.north,region.south]):
        if len(res)>0: delim=','
        res=res+delim+coord_to_str(longitude,latitude)
    return res

# Функция возвращает описание ячейки в формате json 
def region_to_json(region,delim):
    id_=str(int(region.region))
    res=delim+'{\"type\":\"Feature\",\"id\":\"cell'+id_+'\",\"properties\":{\"name\":\"cell'+id_+'\"},\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[['+region_to_str(region)+']]}}'
    return res

# Формируем json файл с описанием сетки
def write_geo_json_data(filename,region):
    f=open(filename,'w')
    f.write('{\"type\":\"FeatureCollection\",\"features\":[')
    delim=''
    for r in region.iterrows():
        f.write(region_to_json(r[1],delim))
        delim=','
    f.write(']}')
    f.close()
    return filename
    
geo_json_data_filename=write_geo_json_data('region.json',region)
geo_json_data =json.load(open(geo_json_data_filename))

Расскраска ячеек сетки на карте будет проводиться на основании значений некой статистики на ячейках сетки.

Понадобится две статистики
1. **Суммарное** количество поездок из ячейки
2. **Среднее** количество поездок из ячейки

In [5]:


# Загружаем агрегированные данные за май
arg_data=pd.read_csv('arg_data_may.csv',sep=';')
#########arg_data.drop(labels='Unnamed: 0',axis=1,inplace=True)
# Строим статистику по среднему количеству поездок
mean_stat=arg_data.mean(axis=0)
mean_stat.index='cell'+mean_stat.index
# Строим статистику по сумме поездок
sum_stat=arg_data.sum(axis=0)
sum_stat.index='cell'+sum_stat.index

In [6]:
sum_stat.head()

celldt    2016-05-01 00:00:002016-05-01 01:00:002016-05-...
cell1                                                     0
cell2                                                     0
cell3                                                     0
cell4                                                     0
dtype: object

Реализуем функцию, которая  рисует карту города, накладывает на неё сетку и ставит метку.

Сетка накладывается исходя из json координат разметки, которые мы сформировали выше. Цвет ячеек сетки зависит от значения статистики функции в ячейке

In [7]:
# Функция возвращает постренную карту
def get_map(geo_json_data,stats,marker):
    # Функция строит карту цветов для отрисовки сетки поверх карты города
    # учитываем, что есть нулевые ячейки - белые
    # карта цветов 3х цветная - зеленый - мало поездок, оранжевый - средне, красный - много поездок
    def get_colormap(data):
        q_1=data[data>0].min()
        q_33=data[data>0].quantile(q=0.33)
        q_66=data[data>0].quantile(q=0.66)
        q_100=data.max()
        return cm.StepColormap(['white','green','orange','red'],index=[0,q_1*0.99,q_33,q_66,q_100],vmin=0,vmax=q_100)
    colormap=get_colormap(stats)
    # Рисуем карту города
    map_ = folium.Map(location=[40.7127, -74.0059],tiles='openstreetmap',zoom_start=10) #tiles='cartodbpositron'tiles = "CartoDB dark_matter"
    # Накладываем сетку
    folium.GeoJson(
        geo_json_data ,style_function=lambda feature: {
            'fillColor': colormap(stats[feature['id']]),
            'weight' : 0
            }).add_to(map_)
    # ставим метку
    folium.Marker(marker[0], popup=marker[1]).add_to(map_)
    return map_

## Выполнение заданий

### 1. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки

In [8]:
(sum_stat==0).sum()

1283

### Карта с сеткой по СУММЕ поездок и меткой Эмпайр-Стейт-Билдинг
$2.$ Нарисуйте статическую карту Нью-Йорка. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.

$3.$ Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.

In [9]:
# Строим карту и накладываем сетку по сумме поездок
marker_esb=[[40.748817, -73.985428],'Empire State Building']
map1=get_map(geo_json_data,sum_stat,marker_esb)
map1.save('map1.html')
#map1

In [10]:
from IPython.display import IFrame

IFrame(src='./map1.html', width=700, height=600)

### Карта с сеткой по СРЕДНЕМУ ЧИСЛУ поездок и меткой Статуя свободы
$4.$ Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится статуя свободы.

$5.$ Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

In [11]:
# Строим карту и накладываем сетку по сумме поездок
marker_sl=[[40.6854, -74.0307],'Statue of Liberty']
map2=get_map(geo_json_data,mean_stat,marker_sl)
map2.save('map2.html')
#map2
IFrame(src='./map2.html', width=700, height=600)

### 6. Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся. Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

In [12]:
type(map2)

folium.folium.Map

In [15]:
mean_stat1=mean_stat
print ((mean_stat>=5).sum())
mean_stat1[mean_stat<5]=0
#print ((mean_stat<=5).sum())
#mean_stat1[mean_stat>5]=0


102


In [17]:
# Строим карту и накладываем сетку по сумме поездок
marker_sl=[[40.6854, -74.0307],'Statue of Liberty']
#mean_stat1[49]=10
map3=get_map(geo_json_data,mean_stat,marker_sl)
map3.save('map3.html')
#map3
IFrame(src='./map3.html', width=700, height=600)

In [60]:
type(mean_stat)

pandas.core.series.Series

In [78]:
df1 = pd.DataFrame(mean_stat)

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2500 entries, cell1 to cell2500
Data columns (total 1 columns):
0    2500 non-null float64
dtypes: float64(1)
memory usage: 39.1+ KB


In [80]:
print ((df1>0).sum())

0    1217
dtype: int64


In [81]:
df1.head(50)

,0
cell1,0.000000
cell2,0.000000
cell3,0.000000
cell4,0.000000
cell5,0.000000
cell6,0.000000
cell7,0.000000
cell8,0.000000
cell9,0.000000
cell10,0.000000


In [102]:
import numpy as np
int_=np.array([ 21.40871158,  33.47986973])

In [104]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))
write_answer_to_file('{} {}'.format(round(int_[0], 3),round(int_[1], 3)),
                                    'ans2_5.txt')

In [105]:
wilson_interval=(0.95010288414112865, 0.95705273772322286)
write_answer_to_file('{} {}'.format(round(wilson_interval[0], 3),
                                   round(wilson_interval[1], 3)), 
                     'answer2_4.txt')

In [106]:
pi_val=0.022076537690726782
write_answer_to_file(pi_val, 
                     'answer2_3.txt')

In [107]:
write_answer_to_file("NO", 
                     'answer2_2.txt')

In [108]:
write_answer_to_file('14061 20087 20087 20087 28118 28118 28118 28118 137019 195712 195712 195712 273957 273957 273957 273957', 
                     'answer2_1.txt')